In [1]:
from torchvision import models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device == 'cuda':
    torch.set_default_device('cuda')
print(device)

class MobileNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.mobilenet = models.mobilenet_v3_small(weights='IMAGENET1K_V1').cuda()

    def forward(self, x):
        return self.mobilenet(x)




model = MobileNet()
print(model)
# Hyper parameters
num_epochs = 40
lr = 0.001


path = "Database2"
transform_rgb = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Create datasets
dataset = ImageFolder(root=path, transform=transform_rgb)
train_dataset_rgb, test_dataset_rgb = random_split(dataset, [0.8, 0.2])


batch_size = 32
train_loader_rgb = DataLoader(train_dataset_rgb, batch_size=batch_size, shuffle=True)
test_loader_rgb = DataLoader(test_dataset_rgb, batch_size=batch_size, shuffle=False)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(num_epochs):
    model.train()
    correct = 0
    for i, (images, labels) in enumerate(train_loader_rgb):
        images, labels = images.to(device), labels.to(device)
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        correct += (torch.max(outputs.data, 1)[1] == labels).float().sum()
    accuracy = 100 * correct / len(train_dataset_rgb)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}, Accuracy: {accuracy}')
    # Test the Model
    model.eval() 
    correct = 0
    total = 0
    for images, labels in test_loader_rgb:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print('Test Accuracy of the model on test images: %.6f%%' % (100.0*correct/total))
            


#Save the Trained Model
torch.save(model.state_dict(),'model.pkl')
torch.cuda.empty_cache()

cuda


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\rapha/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:05<00:00, 19.5MB/s]


MobileNet(
  (mobilenet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [2]:
from tqdm import tqdm

torch.cuda.empty_cache()

class FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.mobilenet = models.mobilenet_v3_small(weights='IMAGENET1K_V1').cuda()
        self.mobilenet.classifier = nn.Identity()
    
    def forward(self, x):
        return self.mobilenet(x)
    
model = FeatureExtractor()

print(model)
# Hyper parameters
num_epochs = 40
lr = 0.001


path = "Database2"
transform_rgb = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Create datasets
dataset = ImageFolder(root=path, transform=transform_rgb)
train_dataset_rgb, test_dataset_rgb = random_split(dataset, [1.0, 0.])


batch_size = 32
train_loader_rgb = DataLoader(train_dataset_rgb, batch_size=batch_size, shuffle=True)
test_loader_rgb = DataLoader(test_dataset_rgb, batch_size=batch_size, shuffle=False)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in tqdm(range(num_epochs)):
    for i, (images, labels) in enumerate(train_loader_rgb):
        images, labels = images.to(device), labels.to(device)
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()


torch.save(model,'mobilenet_no_norm.pt')
torch.cuda.empty_cache()

c:\Users\rapha\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataset.py:414: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "


FeatureExtractor(
  (mobilenet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(

100%|██████████| 40/40 [07:41<00:00, 11.55s/it]


In [3]:
import pandas as pd

# Load model
model = torch.load('mobilenet_no_norm.pt')

path = "Database"
transform_rgb = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Create datasets
dataset = ImageFolder(root=path, transform=transform_rgb)

batch_size = 32
loader_rgb = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Extract features
model.eval()
df=pd.DataFrame()
for i, (images, labels) in tqdm(enumerate(loader_rgb)):
    images, labels = images.to(device), labels.to(device)
    with torch.no_grad():
        output = model(images)
        df=pd.concat([df,pd.DataFrame(output.cpu().numpy())])

print(df.shape)
df.to_csv("features_mobilenet_no_norm.csv",index=False)

159it [01:14,  2.14it/s]


(5060, 1000)


In [4]:
torch.cuda.empty_cache()